In [ ]:
!git clone https://github.com/heysm/answerboxes.git 

Cloning into 'answerboxes'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 17 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), 94.59 MiB | 7.61 MiB/s, done.


In [ ]:
!gdown --id "1ImOLo4ug2BOMvqxbrSffiMWGWxvkE2N0&confirm=t"

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ImOLo4ug2BOMvqxbrSffiMWGWxvkE2N0&confirm=t
To: /content/folds.zip
100% 94.5M/94.5M [00:03<00:00, 31.5MB/s]


In [ ]:
#importing stuffs
import numpy as np
import gc 
import os
import ctypes
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import LeakyReLU
import pandas as pd

sns.set()

In [ ]:
# transfer learning model 
# mobile net V2 model used

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
# pflu activation function
def pflu_activation(x):
    return x*0.5*(1+(x/(1+x**2)**0.5))

#fpflu actiavtion function
def fpflu_activation(x):
    return tf.math.maximum(x,x/(1+x**2)) 

In [ ]:
from keras.models import load_model

path = '/content/answerboxes/models/'

model1 = load_model(path + 'fold1_95_29.h5', custom_objects={'KerasLayer': hub.KerasLayer , 'fpflu_activation' :fpflu_activation , 'pflu_activation' : pflu_activation  })
model2 = load_model(path +'fold2_95_88.h5', custom_objects={'KerasLayer': hub.KerasLayer , 'fpflu_activation' :fpflu_activation , 'pflu_activation' : pflu_activation  })
model3 = load_model( path + 'fold3_93_97.h5', custom_objects={'KerasLayer': hub.KerasLayer , 'fpflu_activation' :fpflu_activation , 'pflu_activation' : pflu_activation  })
model4 = load_model( path + 'fold4_97_45.h5', custom_objects={'KerasLayer': hub.KerasLayer , 'fpflu_activation' :fpflu_activation , 'pflu_activation' : pflu_activation  })
model5 = load_model( path +'fold5_97_20.h5', custom_objects={'KerasLayer': hub.KerasLayer , 'fpflu_activation' :fpflu_activation , 'pflu_activation' : pflu_activation  })

In [ ]:
models = [model1, model2, model3, model4, model5]

In [ ]:
import pathlib

In [ ]:
def get_interpreter(new_model) : 
  converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
  tflite_model = converter.convert()

  tflite_models_dir = pathlib.Path("/tmp/quant_model")
  tflite_models_dir.mkdir(exist_ok=True, parents=True)

  tflite_model_file = tflite_models_dir/"quant_model.tflite"
  tflite_model_file.write_bytes(tflite_model)

  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  tflite_quant_model = converter.convert()
  tflite_model_quant_file = tflite_models_dir/"omr_quant.tflite"
  tflite_model_quant_file.write_bytes(tflite_quant_model)
  interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
  interpreter_quant.allocate_tensors()
  return interpreter_quant

In [ ]:
from zipfile import ZipFile
file_name = "folds.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    #zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

Extracting all the files now...
Done!


In [ ]:
import os
import numpy as np
from PIL import Image

In [ ]:
def get_image_ds(folder_path) : 

  # create an empty list to store the image data
  images = []

  # loop through all files in the image folder
  for filename in os.listdir(folder_path):
      # check if the file is an image
      if filename.endswith(".jpg") or filename.endswith(".png"):
          # open the image file
          img = Image.open(os.path.join(folder_path, filename))
          img = img.resize((224, 224), Image.BILINEAR)
          # convert the image to a numpy array
          img_data = np.array(img, dtype=float)
          # append the image data to the list
          img_data /= 255.0
          images.append(img_data)

  # convert the list of images to a numpy array
  images = np.array(images)
  return images

In [ ]:
import os
import numpy as np
from PIL import Image

# specify the path to the image folder
folder_path = '/content/folds/5/empty'

# create an empty list to store the image data
images = []

# loop through all files in the image folder
for filename in os.listdir(folder_path):
    # check if the file is an image
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # open the image file
        img = Image.open(os.path.join(folder_path, filename))
        img = img.resize((224, 224))
        # convert the image to a numpy array
        img_data = np.array(img, dtype=float)
        # append the image data to the list
        img_data /= 255.0
        images.append(img_data)

# convert the list of images to a numpy array
images = np.array(images)

In [ ]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter, label, images):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for test_image in images:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == label:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

In [ ]:
import time
from datetime import datetime

In [ ]:
def calculate_accuracy() :
  for i in range(5) :
    print(i)
    total_acc = 0
    date_time = datetime.now()
    s = time.mktime(date_time.timetuple())
    for file in [["confirmed",0], ["crossedout",1], ["empty",2]] : 
      folder_path = "/content/folds/{}/{}".format(i+1,file[0]) 
      images = get_image_ds(folder_path)
      interpreter_quant = get_interpreter(models[i])
      acc = evaluate_model(interpreter_quant, file[1], images)
      print(acc)
      total_acc += acc
    print(total_acc/3)
    date_time = datetime.now()
    e = time.mktime(date_time.timetuple())
    print(e-s)
    print()

In [ ]:
import gc
gc.collect()

4818

In [ ]:
calculate_accuracy()

0
0.9705882352941176
0.9029411764705882
0.9926470588235294
0.9553921568627451
157.0

1
0.9794117647058823
0.9058823529411765
0.9955882352941177
0.9602941176470589
166.0

2
0.9735294117647059
0.8235294117647058
1.0
0.9323529411764705
162.0

3
0.9926470588235294
0.913235294117647
0.9955882352941177
0.967156862745098
158.0

4
0.9691176470588235
0.9205882352941176
1.0
0.9632352941176471
159.0

